In [35]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


In [36]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [37]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [38]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [39]:
def sequential_generator(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=2:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [40]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [75]:
train_lines = [line.strip().split("\t") for line in open('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_train.csv') if len(line.strip().split('\t'))==2]
dev_lines = [line.strip().split("\t") for line in open('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv') if len(line.strip().split('\t'))==2]

In [76]:
train_sentences = [x[0] for x in train_lines]
train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [77]:
n_labels =2

In [78]:
f = open("million_post_corpus_train.csv")
x = 0
total=0
for line in f:
    line = line.strip().split('\t')
    if len(line)!=2:
        continue
    if line[1]=='OFFENSE':
        x+=1
    total+=1
print(x)
print(total)

398
2863


In [91]:
f = open("results_million_corpus_training","w")
total_f1 = 0
total_prec =0
total_recall =0
total_acc = 0
max_f1 = 0

for test_number in range(3):
    print("Test number: %d" %(test_number+1))
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
    model = Model(model_input_embedding, model_output)
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    # model.summary()

    
    samples_per_epoch = len(train_sentences)
    epochs = 50
    batch_size = 32
    steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
    # checkpoint = ModelCheckpoint('best_classification_model_million_post_corpus.h5', monitor='val_acc',save_best_only = True, 
#                                  save_weights_only = True)

    for epoch in range(epochs):
        print("Epoch: %d" %(epoch+1))
        model.fit_generator(
            sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_train.csv', batch_size), 
            steps_per_epoch=steps_per_epoch, epochs=1,
    #         validation_data = sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv', batch_size),
    #         validation_steps = math.ceil(len(dev_sentences) / batch_size),
    #         callbacks = [checkpoint]
        )

        testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
        for i in range(len(dev_sentences)):
            testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
        results = model.predict(testset_features)
        
        
        idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
        predLabels = results.argmax(axis=-1)
        devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
        # print(idx2Label)
        # print(predLabels)
        # print(devLabels)
        f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
        r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
        p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
        a = accuracy_score(devLabels, predLabels)
        if max_f1 < f1:
            print("model saved. F1 is %f" %(f1))
            model.save('best_classification_model_million_corpus.h5')
            max_f1 = f1
        text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
        f.write(text)
        total_f1+=f1
        total_prec+=p
        total_acc+=a
        total_recall+=r
        print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
    print("*****************************************************************************")
final_text = "avg_prec: " +str(total_prec/10)+" total_rec: "+str(total_recall/10) +" total_f1: "+str(total_f1/10) +" total_acc: "
+str(total_acc/10)+" \n"
f.write(final_text)
f.close()

Test number: 1
Epoch: 1
Epoch 1/1
90/90 [==============================] - 24s 271ms/step - loss: 0.4162 - acc: 0.8615


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 2
Epoch 1/1
90/90 [==============================] - 20s 225ms/step - loss: 0.3970 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 3
Epoch 1/1
90/90 [==============================] - 21s 234ms/step - loss: 0.3864 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 4
Epoch 1/1
90/90 [==============================] - 21s 234ms/step - loss: 0.3780 - acc: 0.8670
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 5
Epoch 1/1
90/90 [==============================] - 20s 225ms/step - loss: 0.3757 - acc: 0.8670
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 6
Epoch 1/1
90/90 [==============================] - 20s 226ms/step - loss: 0.3656 - acc: 0.8667
model saved. F1 is 0.020000
Test-Data: Prec: 0.500, Rec: 0.010, F1: 0.020, Acc: 0.831
Epoch: 7
Epoch 1/1
90/90 [==============================] - 20s 225ms/step - loss: 0.3514 - acc: 0.8681


90/90 [==============================] - 20s 223ms/step - loss: 0.0192 - acc: 0.9948
Test-Data: Prec: 0.312, Rec: 0.204, F1: 0.247, Acc: 0.789
*****************************************************************************
Test number: 2
Epoch: 1
Epoch 1/1
90/90 [==============================] - 25s 278ms/step - loss: 0.4169 - acc: 0.8649
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 2
Epoch 1/1
90/90 [==============================] - 20s 223ms/step - loss: 0.3943 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 3
Epoch 1/1
90/90 [==============================] - 20s 223ms/step - loss: 0.3896 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 4
Epoch 1/1
90/90 [==============================] - 21s 229ms/step - loss: 0.3816 - acc: 0.8660
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 5
Epoch 1/1
90/90 [==============================] - 20s 226ms/step - loss: 0.3726 - acc: 0.8667
Test-Data: P

90/90 [==============================] - 20s 225ms/step - loss: 0.0255 - acc: 0.9924
Test-Data: Prec: 0.232, Rec: 0.133, F1: 0.169, Acc: 0.779
*****************************************************************************
Test number: 3
Epoch: 1
Epoch 1/1
90/90 [==============================] - 25s 277ms/step - loss: 0.4143 - acc: 0.8597
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 2
Epoch 1/1
90/90 [==============================] - 20s 221ms/step - loss: 0.3944 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 3
Epoch 1/1
90/90 [==============================] - 20s 228ms/step - loss: 0.3858 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 4
Epoch 1/1
90/90 [==============================] - 20s 225ms/step - loss: 0.3799 - acc: 0.8670
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 5
Epoch 1/1
90/90 [==============================] - 20s 221ms/step - loss: 0.3747 - acc: 0.8663
Test-Data: P

90/90 [==============================] - 21s 229ms/step - loss: 0.0195 - acc: 0.9934
Test-Data: Prec: 0.280, Rec: 0.214, F1: 0.243, Acc: 0.774
*****************************************************************************


TypeError: bad operand type for unary +: 'str'

# transfer learning

In [93]:
n_labels =2

In [94]:
def sequential_generator_germeval(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=3:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [95]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt')]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt')]

In [96]:
train_sentences = [x[0] for x in train_lines]
train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [ ]:
f = open("results_pretraining_millioncorpus","w")
total_f1 = 0
total_prec =0
total_recall =0
total_acc = 0
max_f1 = 0

for test_number in range(5):
    print("Test number: %d" %(test_number+1))
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    # model.summary()

    new_model.load_weights('best_classification_model_million_corpus.h5')
    samples_per_epoch = len(train_sentences)
    epochs = 50
    batch_size = 32
    steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
    # checkpoint = ModelCheckpoint('best_classification_model_million_post_corpus.h5', monitor='val_acc',save_best_only = True, 
#                                  save_weights_only = True)

    for epoch in range(epochs):
        print("Epoch: %d" %(epoch+1))
        new_model.fit_generator(
            sequential_generator('/home/gwiedemann/notebooks/OffLang/sample_train.txt', batch_size), 
            steps_per_epoch=steps_per_epoch, epochs=1,
    #         validation_data = sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv', batch_size),
    #         validation_steps = math.ceil(len(dev_sentences) / batch_size),
    #         callbacks = [checkpoint]
        )

        testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
        for i in range(len(dev_sentences)):
            testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
        results = new_model.predict(testset_features)
        
        
        idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
        predLabels = results.argmax(axis=-1)
        devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
        # print(idx2Label)
        # print(predLabels)
        # print(devLabels)
        f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
        r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
        p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
        a = accuracy_score(devLabels, predLabels)
        if max_f1 < f1:
            print("model saved. F1 is %f" %(f1))
            model.save('best_classification_pretrained_millioncorpus.h5')
            max_f1 = f1
        text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
        f.write(text)
        total_f1+=f1
        total_prec+=p
        total_acc+=a
        total_recall+=r
        print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
    print("*****************************************************************************")
final_text = "avg_prec: " +str(total_prec/10)+" total_rec: "+str(total_recall/10) +" total_f1: "+str(total_f1/10) +" total_acc: "
+str(total_acc/10)+" \n"
f.write(final_text)
f.close()

Test number: 1
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 262ms/step - loss: 0.0000e+00 - acc: 0.9998
model saved. F1 is 0.465849
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 2
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 219ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 4
Epoch 1/1
132/132 [==============================] - 29s 220ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 5
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.0000e+00 - acc: 0.9998
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 6
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.0000e+00 - acc: 0.9998
Test-Da

132/132 [==============================] - 29s 219ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 50
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
*****************************************************************************
Test number: 2
Epoch: 1
Epoch 1/1
132/132 [==============================] - 34s 256ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 2
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 216ms/step - loss: 0.0000e+00 - acc: 1.0000
Test-Data: Prec: 0.438, Rec: 0.498, F1: 0.466, Acc: 0.623
Epoch: 4
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 